## **Tutorial:** Post-Processing MITgcm Lagrangian trajectories.
### How to export MITgcm trajectories evaluated in the Irminger Sea from .mat files to .zarr files for use with the LT-Toolbox.

### **Date Created:** 23 / 11 /2023
### **Created By:** Ollie Tooth
### **Contact:** oliver.tooth@seh.ox.ac.uk

In [1]:
# Import relevnat python packages:
import zarr
import scipy.io
import numpy as np
import xarray as xr
import matplotlib.pyplot as plt


In [2]:
# Import MITgcm Lagrangian trajectories .mat file:
filepath = '/home/snapdragon/HadGEM3-GC31-MM/Proj_Future_Pathways/src/Software/lt_toolbox/tutorials/data/particles_output_5km_1000m.mat'
ds_mitgcm = scipy.io.loadmat(filepath)

# Extract vartiable names from dictionary keys:
vars = [key for key in ds_mitgcm.keys()]

# Define dimension variable for NCEI Lagrangian trajectory format (traj x obs):
# NOTE: This is standard output format of Ocean Parcels.
obs = np.arange(ds_mitgcm['timee'].shape[1])
n_obs = ds_mitgcm['timee'].shape[1] 
# NOTE: We could use any variable with dimensions (traj x obs) here:
traj = np.arange(1, ds_mitgcm['f_lats'].shape[0] + 1)
n_traj = ds_mitgcm['f_lats'].shape[0]


# Construct xarray DataSet from ndarrays stored in .mat file:
ds_traj = xr.Dataset(
         data_vars=dict(
         trajectory=(["traj", "obs"], np.tile(traj, (n_obs, 1)).T),
         time=(["traj", "obs"], np.tile(ds_mitgcm['timee'][0, :], (n_traj, 1))),
         lon=(["traj", "obs"], ds_mitgcm['f_lons']),
         lat=(["traj", "obs"], ds_mitgcm['f_lats']),
         z=(["traj", "obs"], ds_mitgcm['f_deps']),
         temp=(["traj", "obs"], ds_mitgcm['f_TT']),
         sal=(["traj", "obs"], ds_mitgcm['f_SS']),
     ),
     coords=dict(
         traj=(["traj"], traj),
         obs=(["obs"], obs),
     ),
     attrs=dict(description="Trial Irminger Current Lagrangian trajectories evaluated using MITgcm regional configuration at 2km horizontal resolution"),
 )

# --- Defining Attributes ---
# Specifying variable attributes according to the NCEI_NetCDF_Trajectory_Template_v2.0.
# See: https://www.nodc.noaa.gov/data/formats/netcdf/v2.0/trajectoryIncomplete.cdl

# trajectory
ds_traj.trajectory.attrs = {
                            'long_name': "water parcel ID",
                            'cf_role': "trajectory_id"
                            }
# time
ds_traj.time.attrs = {
                        'long_name': "time since begining of the simulation",
                        'standard_name': "time",
                        'unit': 'nanoseconds',
                        'calendar': "none"
                        }

# lat
ds_traj.lat.attrs = {
                    'long_name': "latitude",
                    'standard_name': "latitude",
                    'units': "degrees_north"
                    }
# lon
ds_traj.lon.attrs = {
                    'long_name': "longitude",
                    'standard_name': "longitude",
                    'units': "degrees_east"
                    }
# z
ds_traj.z.attrs = {
                    'long_name': "depth",
                    'standard_name': "depth",
                    'units': "meters",
                    }
# NOTE: modify tracer attributes below as required.
# temp
ds_traj.temp.attrs = {
                        'long_name': "temperature",
                        'standard_name': "temperature",
                        'units': "C"
                        }
# sal
ds_traj.sal.attrs = {
                    'long_name': "salinity",
                    'standard_name': "salinity",
                    'units': "PSU"
                    }

# --- Write trajectory DataSet to .zarr store ---
# Define compressor:
compressor = zarr.Blosc(cname="zstd", clevel=3, shuffle=2)
# Define encodings dictionary:
enc = {var: {"compressor": compressor} for var in ds_traj}

# Define output filepath:
out_filedir = "/home/snapdragon/HadGEM3-GC31-MM/Proj_Future_Pathways/src/Software/lt_toolbox/tutorials/data/"
out_filepath = "MITgcm_2km_upper_1000m_example_traj.zarr"

# Write DataSet to .zarr store:
ds_traj.to_zarr(out_filedir+out_filepath, encoding=enc)
